In [1]:
!pip install openai==0.28

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 2.3 MB/s eta 0:00:00


In [2]:
import re
with open('complete_tex.tex','r') as f:
  latex_string = f.readlines()

figure_list = []
Section_headings = {}
for count,lines in enumerate(latex_string):
  # print(lines)
  if re.match(r'\\section', lines):
    head = lines.lstrip(r'\\section{').rstrip().rstrip('}')
    Section_headings[head] = 0
  if re.search(r'\\includegraphics', lines):
    try:
      figure_list.append(lines.split(']{')[1].rstrip().rstrip('}'))
    except:
      pass
print(figure_list)
print(Section_headings)


['Figures/pipeline.png', 'Figures/param_sen.png']
{'Introduction': 0, 'Method': 0, 'Experiments': 0, 'Conclusion': 0}


In [3]:

def prompt_builder(title,page_length,count):
    return f'''
    {{Section Name:  {title},
    Slide Information: {{
      Number of Slides: {page_length},
      Speaker Notes: {{"Slide {{Number}}": 150 words speech for slide}},
      Table: Latex Table to header and row dictionary if present in {title},
      Image: Figure path if it is present in {title} section (possible selections {figure_list}),
      Generative Prompt: Propose a 10 words prompt for simple slide background
    }}
    }}
    '''

user_input = input('Enter the page length for each section: ')
user_input = user_input.split(',')
print(user_input)

for count,key in enumerate(Section_headings):
  Section_headings[key] = user_input[count]

prompt_base = ''
for count,(key,val) in enumerate(Section_headings.items()):
  prompt_base+=prompt_builder(key,val,count+1)

print(prompt_base)

Enter the page length for each section: 3,2,1,1
['3', '2', '1', '1']

    {Section Name:  Introduction,
    Slide Information: {
      Number of Slides: 3,
      Speaker Notes: {"Slide {Number}": 150 words speech for slide},
      Table: Latex Table to header and row dictionary if present in Introduction,
      Image: Figure path if it is present in Introduction section (possible selections ['Figures/pipeline.png', 'Figures/param_sen.png']),
      Generative Prompt: Propose a 10 words prompt for simple slide background
    }
    }
    
    {Section Name:  Method,
    Slide Information: {
      Number of Slides: 2,
      Speaker Notes: {"Slide {Number}": 150 words speech for slide},
      Table: Latex Table to header and row dictionary if present in Method,
      Image: Figure path if it is present in Method section (possible selections ['Figures/pipeline.png', 'Figures/param_sen.png']),
      Generative Prompt: Propose a 10 words prompt for simple slide background
    }
    }
    
    

In [29]:
messages=[
    {
      "role": "system",
      "content": "You are an expert slide expert that extracts latex papers and create presentation slides.",
    },
    {
      "role": "user",
      "content": f'''
      Given the document: {latex_string}
      Analyse paper by section and strictly follow this template but json format:
      {prompt_base}
      '''
    }
  ]
print(messages)

[{'role': 'system', 'content': 'You are an expert slide expert that extracts latex papers and create presentation slides.'}, {'role': 'user', 'content': '\n      Given the document: [\'\\\\documentclass[sigconf,nonacm]{acmart}\\n\', \'\\\\makeatletter\\n\', \'\\\\renewcommand\\\\@formatdoi[1]{\\\\ignorespaces}\\n\', \'\\\\makeatother\\n\', \'\\n\', \'\\n\', \'\\\\AtBeginDocument{\\\\providecommand\\\\BibTeX{{\\\\normalfont B\\\\kern-0.5em{\\\\scshape i\\\\kern-0.25em b}\\\\kern-0.8em\\\\TeX}}}\\n\', \'\\n\', \'\\n\', \'\\\\acmDOI{}\\n\', \'\\\\acmISBN{}\\n\', \'\\n\', \'\\n\', \'\\\\usepackage{hyperref}\\n\', \'\\\\usepackage{siunitx}\\n\', \'\\\\usepackage{xspace}\\n\', \'\\\\usepackage{multirow}\\n\', \'\\\\usepackage{enumitem}\\n\', \'\\\\usepackage{placeins}\\n\', \'\\\\usepackage{natbib}\\n\', \'\\\\newcommand{\\\\ours}{\\\\textsc{TinyLLM}\\\\xspace}\\n\', \'\\\\settopmatter{printacmref=false}\\n\', \'\\\\setcopyright{none}\\n\', \'\\\\pagestyle{plain}\\n\', \'\\n\', \'\\n\', \'\\

In [30]:
import openai
import os

openai.api_key  = ''

def get_completion_from_messages(messages,
                                 model="gpt-4-turbo-preview",
                                 response_format={ "type": "json_object" },
                                 temperature=0, max_tokens=1200):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens,
    )
    return response.choices[0].message.content

In [31]:
output=get_completion_from_messages(messages=messages)

In [32]:
import re
import json

def LLM_Output_to_dict(LLM_Output,Content_Num):
    Section_Json = []
    for x in LLM_Output.split('\n\n'):
        Section_Json.append(x.replace('```','').lstrip().rstrip().lstrip('json'))
    Section_Json = [x for x in Section_Json if not x == '']
    result = json.loads(f'{Section_Json[Content_Num]}')
    Section_Name = result['Section Name']
    Number_of_Slides = result['Slide Information']['Number of Slides']
    Slide_Info = [x for x in result['Slide Information']['Speaker Notes'].values()]
    Table = result['Slide Information']['Table']
    Image = result['Slide Information']['Image']
    Generative_Prompt = result['Slide Information']['Generative Prompt']
    return Section_Name,Number_of_Slides,Slide_Info,Table,Image,Generative_Prompt



Section_Name,Number_of_Slides,Speaker_Notes,Table,Image,Generative_Prompt = LLM_Output_to_dict(output,0)
print(Section_Name)
print(Number_of_Slides)
print(Speaker_Notes)
print(Table)
print(Image)
print(Generative_Prompt)



Introduction
3
['Today, we embark on an exploration of a groundbreaking approach in the realm of language models, specifically focusing on the transition of knowledge from large to small language models. The motivation behind this research stems from the desire to harness the power of large language models in a more efficient and cost-effective manner. By transferring capabilities from larger models to smaller ones, we aim to achieve high performance without the associated computational overhead.', 'However, the journey is not without its challenges. Existing methods of knowledge distillation, while effective to a degree, face significant limitations in terms of knowledge diversity and the richness of contextual information. This presentation introduces TinyLLM, a novel paradigm designed to overcome these hurdles by leveraging multiple large language models as teachers, thereby enriching the student model with diverse knowledge and deep contextual insights.', "The essence of TinyLLM li

In [33]:
def bullet_points(prompt):
  bullets = []
  for info in prompt:
    output = get_completion_from_messages(messages=[
        {
          "role": "system",
          "content": "You are an expert presentation designer that create bullet points from speech",
        },
        {
          "role": "user",
          "content": f'''Given the document: {info}
          Give me 4 concise, 10 words bullet points for the slide''',
        }
      ],model='gpt-3.5-turbo',)
    bullets.append([x.lstrip('* ').rstrip() for x in output.split('\n')])
  return bullets
for x in bullet_points(Speaker_Notes):
  print(x)

['1. Transitioning knowledge from large to small language models efficiently.', '2. Harnessing power of large models in a cost-effective manner.', '3. Achieving high performance without excessive computational overhead.', '4. Exploring groundbreaking approach in language model transition for efficiency.']
['- Challenges in existing knowledge distillation methods: diversity and contextual limitations.', '- Introducing TinyLLM: innovative paradigm using multiple large language models.', '- TinyLLM enriches student model with diverse knowledge and deep insights.', '- Overcoming hurdles by leveraging large language models as teachers.']
["- TinyLLM: Innovative knowledge distillation approach emphasizing 'why' and 'how'.", '- Deeper comprehension leads to improved reasoning and performance.', '- Focus on rationales behind correct answers enhances student model understanding.', '- Enhanced reasoning capabilities across datasets and tasks with TinyLLM.']


Stable Diffusion

In [ ]:
!pip install stability-sdk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.1/117.1 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.8/409.8 kB 34.5 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.62.0
    Uninstalling grpcio-1.62.0:
      Successfully uninstalled grpcio-1.62.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-metadata 1.14.0 requires protobuf<4.21,>=3.20.3, but you have protobuf 4.21.12 which is incompatible.


In [ ]:
import io
import warnings
from PIL import Image
from stability_sdk import client
import stability_sdk.interfaces.gooseai.generation.generation_pb2 as generation

In [ ]:
stability_api = client.StabilityInference(
    key=os.environ['STABILITY_KEY'], # API Key reference.
    verbose=True, # Print debug messages.
    engine="stable-diffusion-xl-1024-v1-0", # Set the engine to use for generation.
    # Check out the following link for a list of available engines: https://platform.stability.ai/docs/features/api-parameters#engine
)

INFO:stability_sdk.client:Opening channel to grpc.stability.ai:443
INFO:stability_sdk.client:Channel opened to grpc.stability.ai:443


In [ ]:
answers = stability_api.generate(
    prompt= Generative_Prompt,
    seed=4253978046, # If a seed is provided, the resulting generated image will be deterministic.
                     # What this means is that as long as all generation parameters remain the same, you can always recall the same image simply by generating it again.
                     # Note: This isn't quite the case for Clip Guided generations, which we'll tackle in a future example notebook.
    steps=50, # Amount of inference steps performed on image generation. Defaults to 30.
    cfg_scale=8.0, # Influences how strongly your generation is guided to match your prompt.
                   # Setting this value higher increases the strength in which it tries to match your prompt.
                   # Defaults to 7.0 if not specified.
    width=1024, # Generation width, defaults to 512 if not included.
    height=1024, # Generation height, defaults to 512 if not included.
    samples=1, # Number of images to generate, defaults to 1 if not included.
    sampler=generation.SAMPLER_K_DPMPP_2M # Choose which sampler we want to denoise our generation with.
                                                 # Defaults to k_dpmpp_2m if not specified. Clip Guidance only supports ancestral samplers.
                                                 # (Available Samplers: ddim, plms, k_euler, k_euler_ancestral, k_heun, k_dpm_2, k_dpm_2_ancestral, k_dpmpp_2s_ancestral, k_lms, k_dpmpp_2m, k_dpmpp_sde)
)

In [ ]:
for resp in answers:
    for artifact in resp.artifacts:
        if artifact.finish_reason == generation.FILTER:
            warnings.warn(
                "Your request activated the API's safety filters and could not be processed."
                "Please modify the prompt and try again.")
        if artifact.type == generation.ARTIFACT_IMAGE:
            img = Image.open(io.BytesIO(artifact.binary))
            img.save(str(artifact.seed)+ ".png")

INFO:stability_sdk.client:Sending request.
INFO:stability_sdk.client:Got answer  with artifact types ['ARTIFACT_IMAGE'] in 8.80s


Powerpoint Pipeline

In [22]:
!pip install python-pptx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 kB 15.5 MB/s eta 0:00:00


In [47]:
Section_Name = 'Introduction'
Number_of_Slides = 3
Speaker_Notes = ['Today, we embark on an exploration of a groundbreaking approach in the realm of language models, specifically focusing on the transition of knowledge from large to small language models. The motivation behind this research stems from the desire to harness the power of large language models in a more efficient and cost-effective manner. By transferring capabilities from larger models to smaller ones, we aim to achieve high performance without the associated computational overhead.', 'However, the journey is not without its challenges. Existing methods of knowledge distillation, while effective to a degree, face significant limitations in terms of knowledge diversity and the richness of contextual information. This presentation introduces TinyLLM, a novel paradigm designed to overcome these hurdles by leveraging multiple large language models as teachers, thereby enriching the student model with diverse knowledge and deep contextual insights.', "The essence of TinyLLM lies in its innovative approach to knowledge distillation, which not only focuses on the correct answers but also delves into the rationales behind these answers. By understanding the 'why' and 'how', the student model gains a deeper comprehension of the subject matter, leading to improved reasoning capabilities and performance across various datasets and tasks."]
Table = None
Image = 'Figures/pipeline.png'
Generative_Prompt = 'Innovative knowledge distillation paradigm visualization'

In [23]:
from pptx import Presentation
from pptx.util import Inches


class PresentationCreator:

    def __init__(self) -> None:
        self.prs = Presentation()
        # Define slide layout mappings
        self.layout_mappings = {
            "1": self.prs.slide_layouts[0],  # Title Slide
            "2": self.prs.slide_layouts[1],  # Title and Content
            "9": self.prs.slide_layouts[8],  # Picture with Caption
            # Add more mappings as needed for other layouts
        }

    def add_slide(self, slide_info: dict) -> None:
        """Add slide to presentation."""
        # Extract slide info
        layout_index = slide_info.get("layout", "2")
        if layout_index == "1":
            self.add_title_layout(slide_info)
        elif layout_index == "2":
            self.add_title_content_layout(slide_info)
        elif layout_index == "9":
            self.add_picture_caption_layout(slide_info)

    def add_title_layout(self, slide_info: dict) -> None:
        """Add slide with title layout."""
        slide = self.prs.slides.add_slide(self.prs.slide_layouts[0])
        title_shape = slide.shapes.title
        title_shape.text = slide_info["title"]

    def add_title_content_layout(self, slide_info: dict) -> None:
        """Add slide with title and content layout."""
        slide = self.prs.slides.add_slide(self.prs.slide_layouts[1])
        title_shape = slide.shapes.title
        title_shape.text = slide_info["title"]
        content_shape = slide.placeholders[1]
        content_frame = content_shape.text_frame
        content_frame.text = slide_info["content"]

    def add_picture_caption_layout(self, slide_info: dict) -> None:
        """Add slide with picture and caption layout."""
        slide = self.prs.slides.add_slide(self.prs.slide_layouts[8])
        title_shape = slide.shapes.title
        title_shape.text = slide_info["title"]
        # title_shape.left = Inches(2)
        # title_shape.text_frame.paragraphs[0].font.size = Inches(0.25)
        left = Inches(1)  # Adjust left position as needed
        top = Inches(1.5)  # Adjust top position as needed
        width = Inches(5)  # Adjust width as needed
        height = Inches(3)  # Adjust height as needed
        slide.shapes.add_picture(slide_info["image"], left, top, width, height)

    def save_presentation(self, file_name: str) -> None:
        """Save presentation to file."""
        self.prs.save(file_name)

    def convert_presentation_to_image(self, file_name: str) -> None:
        """Convert presentation to image."""
        pass


In [61]:
Presentation_Object = PresentationCreator()
Layout = '2'
bullets = bullet_points(Speaker_Notes)
Slide_info = {'layout':Layout,'title':Section_Name,'content':'\n'.join(bullets[0]),'image':Image}
print(Slide_info)
Presentation_Object.add_slide(Slide_info)
Presentation_Object.add_picture_caption_layout(Slide_info)

{'layout': '2', 'title': 'Introduction', 'content': '1. Transitioning knowledge from large to small language models efficiently.\n2. Harnessing power of large models in cost-effective manner.\n3. Achieving high performance without computational overhead through knowledge transfer.\n4. Exploring groundbreaking approach in language models for efficiency and effectiveness.', 'image': 'Figures/pipeline.png'}


In [62]:
Presentation_Object.save_presentation('test.pptx')